In [1]:
file = open("./fichier.txt")
try:
    for line in file:
        print(line.strip())
finally:
    file.close()

Line 1 / 5
Line 2 / 5
Line 3 / 5
Line 4 / 5
Line 5 / 5
END


In [2]:
with open("./fichier.txt") as file:
    for line in file:
        print(line.strip())
    

Line 1 / 5
Line 2 / 5
Line 3 / 5
Line 4 / 5
Line 5 / 5
END


In [12]:
class DemonstrationContext():
    def __enter__(self):
        print("J'entre dans le contexte")
        return "Return enter"
    def __exit__(self, exc_type, exc_value, traceback):
        print("Je quitte le contexte")
        if (exc_type is None):
            print("no problem")
        else:
            print(f"Error with {exc_value}")
            print(f"{traceback}")
            print(f"{traceback.tb_frame}")
        return True

with DemonstrationContext() as string_enter:
    print(string_enter)
    raise TypeError("Prombleme dans le code")
    
        

J'entre dans le contexte
Return enter
Je quitte le contexte
Error with Prombleme dans le code
<frame at 0x00000185FD83F7C0, file 'C:\\Users\\PaulE\\AppData\\Local\\Temp\\ipykernel_32800\\3070832516.py', line 15, code <module>>


In [14]:
class Config:
    etat = "normal"

config = Config()
print(config.etat)
etat_currant = config.etat
try:
    config.etat = "temporatoire"
    print("USe quelqueschoese")
    print(config.etat)
finally:
    config.etat = etat_currant

print(config.etat)


normal
USe quelqueschoese
temporatoire
normal


In [16]:
class ConfigManager:
    def __init__(self, currentConfig, etatTemporaireString):
        self.currentConfig = currentConfig
        self.saveTemporaireString = currentConfig.etat
        self.etatTemporaireString = etatTemporaireString
    def __enter__(self):
        self.currentConfig.etat = self.etatTemporaireString
        print("Actif")
    def __exit__(self, exc_type, exc_value, traceback):
        self.currentConfig.etat = self.saveTemporaireString 
        print("rEstauré")
        
config = Config()
print(config.etat)
with ConfigManager(config, "17h30"):
    print(" a lintérieur ")
with ConfigManager(config, "18h30"):
    print(" a lintérieur ")

normal
Actif
 a lintérieur 
rEstauré
Actif
 a lintérieur 
rEstauré


In [18]:
from contextlib import contextmanager

config = Config()
print(config.etat)

@contextmanager
def mettre_etat_temporaire(currentConfig, etatTemporaire):
    saveEtatTemporaire = currentConfig.etat
    currentConfig.etat = etatTemporaire
    print("Actif")
    try:
        yield
    except Exception as e:
        print(f"{e}")
    else:
        print("No erro")
    finally:
        currentConfig.etat = saveEtatTemporaire
        print("rEstauré")
        
with mettre_etat_temporaire(config, "17h30"):
    print(" a lintérieur ")
    
with mettre_etat_temporaire(config, "18h30"):
    print(" a lintérieur ")
        
    

normal
Actif
 a lintérieur 
No erro
rEstauré
Actif
 a lintérieur 
No erro
rEstauré


In [24]:
from contextlib import suppress

with suppress(FileNotFoundError), open("./fichier_introuvable.txt") as file:
    for line in file:
        print(line.strip())
    

In [34]:
from contextlib import redirect_stdout, redirect_stderr
import sys
import io

with io.StringIO() as buf, redirect_stdout(buf):
    print("Valeur à redirigé")
    output = buf.getvalue()
        
print(f" Output : {output}")

with io.StringIO() as buf, redirect_stderr(buf):
    print("Error à redirigé", file=sys.stderr)
    output = buf.getvalue()
        
print(f" Output : {output}")

 Output : Valeur à redirigé

 Output : Error à redirigé



In [42]:
from contextlib import closing
class FakeDbConnection:
    def __init__(self):
        print("Preparation du context de la DB")
    def close(self):
        print("Nettoyage du contexte de la DB")
    def send(self, value):
        print(value)
"""
connection = FakeDbConnection()
try:
    connection.send("Envoir donnée à la DB")
finally:
    connection.close()
"""
with closing(FakeDbConnection()) as connection:
    connection.send("Envoir donnée à la DB")

Preparation du context de la DB
Envoir donnée à la DB
Nettoyage du contexte de la DB


In [52]:
from requests import Session
class HTTPSession:      
    def __enter__(self):
        self.s = Session()
        print("Connection")
        return self
    def __exit__(self, ex_type, ex_value, ex_traceback):
        self.s.close()
        print("Connection terminée")
        return True
        
with HTTPSession() as hs:
    response = hs.s.get("https://icanhazdadjoke.com/", headers={'Accept' : 'text/plain'})
    raise Exception
    if response.status_code == 200:
        print(f"La blague : {response.text}")
    else :
        print("Pas de blague")

        



Connection
Connection terminée


In [54]:
        
@contextmanager
def http_session():
    session = Session()
    try:
        yield session
    finally:
        session.close()
        
with http_session() as hs:
    response = hs.get("https://icanhazdadjoke.com/", headers={'Accept' : 'text/plain'})
    if response.status_code == 200:
        print(f"La blague : {response.text}")
    else :
        print("Pas de blague")

La blague : What do you call cheese by itself? Provolone.


In [55]:
with closing(Session()) as hs:
    response = hs.get("https://icanhazdadjoke.com/", headers={'Accept' : 'text/plain'})
    if response.status_code == 200:
        print(f"La blague : {response.text}")
    else :
        print("Pas de blague")

La blague : Why couldn't the kid see the pirate movie? Because it was rated arrr!


# pytest

In [74]:
import pytest

def diviser(a, b):
    if b == 0:
        raise ValueError("Try to divide by zero!")
    return a / b

def test_div_normal():
    assert diviser(10, 2) == 5
    assert diviser(10, -2) == -5

def test_div_by_zero():
    with pytest.raises(ValueError) as exc:
        diviser(10, 0) 
    assert "Zero" in str(exc.value)
test_div_normal()
test_div_by_zero()

AssertionError: 